This notebook is the preprocessing code for SciPlex3 dataset. We have provided processed data in https://drive.google.com/drive/folders/1QWjmpYZMaqxfLwIeLjwoz-H9vX60udeu?usp=drive_link for you to use directly.

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
from CRISP.utils import rank_genes_groups_by_cov
import CRISP.scFM as scFM

### Load and preprocess raw data

In [ ]:
DATA_DIR = "raw/data/path" # we use raw SciPlex3 data provided by chemCPA
adatas = []
for i in range(5):
    adatas.append(sc.read(DATA_DIR+'datasets/'+f'sciplex_raw_chunk_{i}.h5ad'))
adata = adatas[0].concatenate(adatas[1:])

In [7]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)

In [13]:
import re
def remove_special_characters(input_string):
    cleaned_string = re.sub(r'[^a-zA-Z0-9\s\(\)\-\+]', '', input_string)
    return cleaned_string

In [14]:
adata.obs['condition'] = adata.obs.product_name.copy()
adata.obs['condition'] = [x.split(' ')[0] for x in adata.obs['condition']]
adata.obs["condition"] = adata.obs["condition"].astype('category').cat.rename_categories({"(+)-JQ1": "JQ1"})
adata.obs["condition"] = adata.obs["condition"].cat.rename_categories({"Vehicle": "control"})
adata.obs['condition'] = adata.obs['condition'].apply(remove_special_characters)

In [15]:
adata.obs['dose_val'] = adata.obs['dose'].astype(float) / np.max(adata.obs['dose'].astype(float))
adata.obs['drug_dose_name'] = adata.obs.condition.astype(str) + '_' + adata.obs.dose_val.astype(str)
adata.obs['cov_drug_dose_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.drug_dose_name.astype(str)
adata.obs['cov_drug_name'] = adata.obs.cell_type.astype(str) + '_' + adata.obs.condition.astype(str)
adata.obs['control'] = [1 if x == 'control' else 0 for x in adata.obs.condition.values]

In [ ]:
import re
def remove_parentheses(input_string):
    result = re.sub(r'\([^)]*\)', '', input_string)
    return result.strip() 
import pubchempy as pcp
drug_names = list(np.unique(adata.obs['condition'].values))
drug_smiles_dict = {}
for i in range(len(drug_names)):
    drug = drug_names[i]
    mol = drug
    if drug == 'MC1568':
        mol = 'MC-1568'
    if drug == 'Rucaparib (AG-014699PF-01367338) phosphate':
        mol = 'Rucaparib phosphate'
    print(drug, mol)
    try:
        compound = pcp.get_compounds(mol, 'name')[0]
    except:
        mol = remove_parentheses(mol)
        compound = pcp.get_compounds(mol, 'name')[0]
    smiles = compound.to_dict(properties=['canonical_smiles'])['canonical_smiles']
    drug_smiles_dict[drug] = smiles

In [14]:
import pickle
# load dict of drug:SMILES
with open('drug_dict.pkl','wb') as f:
    drug_dict = pickle.load(f)

In [15]:
adata.obs['SMILES'] = adata.obs.condition.map(drug_dict)

In [20]:
(adata.obs.condition=='nan').sum()

0

In [21]:
adata.obs["SMILES"] = adata.obs["SMILES"].astype("category").cat.rename_categories({"": "CS(C)=O"})

In [23]:
from rdkit import Chem
adata.obs.SMILES = adata.obs.SMILES.apply(Chem.CanonSmiles)

In [ ]:
model_path = '/path/to/scGPT/model' # use cancer
adata = scFM.calc_gpt(adata,model_path,gene_name='gene_name',return_key='X_scGPT')

### generate train-test-ood split

In [5]:
def split_dataset(adata,cell_types_inood,split_key):
    # set all ood cell type samples as ood
    adata.obs[split_key] = 'train'
    setout_idx = adata[adata.obs.cell_type.isin(cell_types_inood)].obs.index
    adata.obs[split_key].loc[setout_idx] = 'ood'

    # set 20% left samples as test
    def stratified_sample(group):
        return group.sample(frac=0.2) 

    settest_idx = adata[adata.obs[split_key] != 'ood'].obs.groupby(['cell_type','condition'], group_keys=False).apply(stratified_sample).index
    adata.obs[split_key].loc[settest_idx] = 'test'

    # set 75% unperturbed ood cell type samples as train
    def stratified_sample(group):
        return group.sample(frac=0.75)
    settrain_idx = adata[(adata.obs[split_key] == 'ood') & (adata.obs.control == 1)].obs.groupby(['cell_type','condition'], group_keys=False).apply(stratified_sample).index
    adata.obs[split_key].loc[settrain_idx] = 'train'

    return adata

In [22]:
def split_dataset_drugs(adata,drugs_inood, celltype_inood, split_key):

    # set all perturbed samples in ood cell type and drugs as ood
    adata.obs[split_key] = 'train'
    setout_idx = adata[(adata.obs.condition.isin(drugs_inood) | adata.obs.cell_type.isin(celltype_inood)) & (adata.obs.control==0)].obs.index
    adata.obs[split_key].loc[setout_idx] = 'ood'

    # set 15% left samples as test
    def stratified_sample(group):
        return group.sample(frac=0.15) 

    settest_idx = adata[adata.obs[split_key] != 'ood'].obs.groupby(['cell_type','condition'], group_keys=False).apply(stratified_sample).index
    adata.obs[split_key].loc[settest_idx] = 'test'

    # set 20% unperturbed samples in train set as ood
    all_celltype = set(adata[adata.obs[split_key]!='ood'].obs.cell_type.values)
    def stratified_sample(group):
        return group.sample(frac=0.2)
    settrain_idx = adata[(adata.obs.cell_type.isin(all_celltype)) & (adata.obs.control == 1) & (adata.obs[split_key]=='train')].obs.groupby(['cell_type','condition'], group_keys=False).apply(stratified_sample).index
    adata.obs[split_key].loc[settrain_idx] = 'ood'

    return adata

In [7]:
all_drugs = np.unique(adata.obs['condition'].values)
len(all_drugs)

188

In [5]:
selected = ['Dacinostat', 
            'Givinostat', 
            'Belinostat', 
            'Hesperadin', 
            'Quisinostat', 
            'Alvespimycin', 
            'Tanespimycin', 
            'TAK-901', 
            'Flavopiridol']

In [ ]:
adata = split_dataset(adata,['A549'],'split')
adata = split_dataset(adata,['K562'],'split2')
adata = split_dataset(adata,['MCF7'],'split3')

adata = split_dataset(adata,selected,['MCF7'],'split_drugs')
adata = split_dataset(adata,selected,['A549'],'split_drugs2')
adata = split_dataset(adata,selected,['K562'],'split_drugs3')

In [6]:
pd.crosstab(adata[adata.obs['control']==0].obs['split3'],adata[adata.obs['control']==0].obs['cell_type'])

cell_type,A549,K562,MCF7
split3,,,
ood,0,0,285652
test,27947,28681,0
train,111781,114712,0


In [7]:
pd.crosstab(adata[adata.obs['control']==0].obs['split2'],adata[adata.obs['control']==0].obs['cell_type'])

cell_type,A549,K562,MCF7
split2,,,
ood,0,143393,0
test,27947,0,57133
train,111781,0,228519


In [8]:
pd.crosstab(adata[adata.obs['control']==0].obs['split'],adata[adata.obs['control']==0].obs['cell_type'])

cell_type,A549,K562,MCF7
split,,,
ood,139728,0,0
test,0,51641,102959
train,0,91752,182693


### create sub adata which contains top 5000 hvg

In [17]:
sc.pp.highly_variable_genes(adata, n_top_genes=5000, subset=False)

In [20]:
adata_sub = adata[:, (adata.var.highly_variable)].copy()

In [ ]:
rank_genes_groups_by_cov(adata_sub, groupby='cov_drug_dose_name', covariate='cell_type', control_group='control_0.0')

In [9]:
len(adata_sub.uns['rank_genes_groups_cov'])

2244

In [ ]:
adata.write('path/to/sciplex_smiles_hvgenes_scgpt_resplit.h5ad')

### create smiles df and calc rdkit smiles embeddinig

We have provided calculated smiles embedding in /data/drug_embeddings folder. You can skip it in practice when reproduce experimental results

In [4]:
drug_names = set(adata.obs['condition'])
drugs_names_unique_sorted = np.array(sorted(drug_names))
name_to_smiles_map = {
    drug: smiles
    for drug, smiles in adata.obs.groupby(
        ['condition', 'SMILES']
    ).groups.keys()
}
drug_smile_sorted = [name_to_smiles_map[name] for name in drugs_names_unique_sorted]      


In [24]:
import numpy as np
import pandas as pd
smiles_df = pd.DataFrame({'smiles':drug_smile_sorted})
smiles_df.to_csv('path/to/smiles/df')

In [110]:
smiles_df_lincs = pd.read_csv('path/to/lincs/smiles/df',index_col=0)
smiles_list_lincs = smiles_df_lincs['smiles'].values
smiles_union = list(set(drug_smile_sorted).union(set(smiles_list_lincs)))
smiles_union_df = pd.DataFrame({'smiles':list(smiles_union)})
smiles_union_df.to_csv('path/to/merged/smiles/df')

In [132]:
import pandas as pd
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

smiles_list = smiles_union_df['smiles'].values

In [ ]:
from descriptastorus.descriptors.DescriptorGenerator import MakeGenerator
generator = MakeGenerator(("RDKit2D",))

In [ ]:
n_jobs = 16
data = Parallel(n_jobs=n_jobs)(delayed(generator.process)(smiles) for smiles in tqdm(smiles_list, position=0, leave=True) )

In [ ]:
embedding = np.array(data)

In [ ]:
drug_idx, feature_idx = np.where(np.isnan(embedding))

In [138]:
drug_idx_infs, feature_idx_infs = np.where(np.isinf(embedding))

drug_idx = np.concatenate((drug_idx, drug_idx_infs))
feature_idx = np.concatenate((feature_idx, feature_idx_infs))
embedding[drug_idx, feature_idx] = 0

In [139]:
import pandas as pd

df = pd.DataFrame(data=embedding,index=smiles_list,columns=[f'latent_{i}' for i in range(embedding.shape[1])]) 

# Drop first feature from generator (RDKit2D_calculated)
df.drop(columns=['latent_0'], inplace=True)

# Drop columns with 0 standard deviation
threshold = 0.01
columns=[f'latent_{idx+1}' for idx in np.where(df.std() <= threshold)[0]]
print(f'Deleting columns with std<={threshold}: {columns}')
df.drop(columns=[f'latent_{idx+1}' for idx in np.where(df.std() <= 0.01)[0]], inplace=True)

Deleting columns with std<=0.01: ['latent_90', 'latent_103', 'latent_152', 'latent_164', 'latent_187', 'latent_196']


In [ ]:
normalized_df=(df-df.mean())/df.std()

In [141]:
normalized_df.to_parquet('path/to/drug/embedding')